In [14]:
import pickle
import numpy as np
import pandas as pd

import faiss

from gensim.models import Word2Vec

In [15]:
# data path
DATA_PATH = 'res/data/new_feat_df.csv'

arr_title = pd.read_csv(DATA_PATH, usecols=['title']).values
cleaned_arr_title = pd.read_csv(DATA_PATH, usecols=['cleaned_title']).fillna('').values
arr_weight = pd.read_csv(DATA_PATH, usecols=['weight']).values

In [16]:
corpus_cleaned_title = cleaned_arr_title.flatten().tolist()
splited_cleaned_arr_title = [title.split(' ')for title in corpus_cleaned_title]

## 1. Make word2vec model

In [ ]:
w2v_model = Word2Vec(splited_cleaned_arr_title, size=100, window=5, min_count=1, workers=4)
w2v_model.save('res/model/가공식품_w2v.model')

## 2. Make tfidf model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus_cleaned_title)

pickle.dump(vectorizer, open('res/model/tfidf.pkl', "wb"))

## 3. WordVectorSummation

In [ ]:
from wvs_generator import WVSGenerator

# Word2vec & TFIDF model path
W2V_PATH = 'res/model/가공식품_word2vec.model'
TFIDF_PATH = 'res/model/가공식품_tfidf_model.pkl'

w2v_model = Word2Vec.load(W2V_PATH)
tfidf_model = pickle.load(open(TFIDF_PATH, 'rb'))

In [ ]:
vector_generator = WVSGenerator(arr_title, cleaned_arr_title, w2v_model, tfidf_model)

# make wvs
vector_generator.do(n_worker=3)

# save total wvs
vector_generator.save_total_wvs('res/data/가공식품_wvs.npz')

## 4. Make faiss index

In [7]:
from index_handle import IndexHandler

# vector path
arr_title_vector_path = 'res/data/가공식품_wvs.npz'
arr_img_vector_path = 'res/data/가공식품_IMGF.npz'
arr_concat_vector_path = 'res/data/CONCAT_가공식품.npz'


# load vector (title / img / concat)
arr_title_vector = np.load(arr_title_vector_path)['arr_0']
arr_img_vector = np.load(arr_img_vector_path)['arr_0']
arr_concat_vector = np.load(arr_concat_vector_path)['arr_0']

# make concat vector 
# contig_query_vector = np.ascontiguousarray(query_vector, dtype=np.float32)
# contig_img_vector = np.ascontiguousarray(img_vector, dtype=np.float32)
# concat_vector = np.hstack((contig_query_vector, contig_img_vector))
# np.savez('res/data/CONCAT_가공식품', concat_vector)


# index path
title_index_path = 'res/model/index/title.h5'
img_index_path = 'res/model/index/image.h5'
concat_index_path = 'res/model/index/concat.h5'

# index_handler
index_handler = IndexHandler()

# make & save index
index_handler.make_index(arr_title_vector)
index_handler.save_index(title_index_path)

index_handler.make_index(arr_img_vector)
index_handler.save_index(img_index_path)

index_handler.make_index(arr_concat_vector)
index_handler.save_index(concat_index_path)

True
True
True


In [17]:
from index_handle import IndexHandler
index_handler = IndexHandler()

# load index
title_index = index_handler.load_index(title_index_path)
img_index = index_handler.load_index(img_index_path)
concat_index = index_handler.load_index(concat_index_path)

## 5-1. Intersection filter

In [18]:
from filter_handle import FilterHandler
filter_handler = FilterHandler(arr_title, arr_weight, title_index, arr_title_vector, img_index, arr_img_vector, concat_index, arr_concat_vector)

In [19]:
# 미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 300g
query_title = arr_title[15000]

prod_filter = filter_handler.get_filter('intersection')
sim_titles = prod_filter.do(query_title)

print('제목 : {0}'.format(query_title[0]))
print('-'*50)
print('유사 상품 제목')
print(sim_titles.flatten())

제목 : 미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 300g
--------------------------------------------------
유사 상품 제목
['미소반 깔끔향긋 방풍나물절임 300g 2통' '미소반 깔끔향긋 방풍나물절임 300g 3통'
 '미소반 깔끔향긋 방풍나물절임 300g 5통' '미소반 깔끔향긋 방풍나물절임 300g 5통'
 '미소반 깔끔향긋 방풍나물절임 300g 3통' '미소반 깔끔향긋 방풍나물절임 300g 2통'
 '미소반 깔끔향긋 방풍나물절임 300g 3통' '미소반 깔끔향긋 방풍나물절임 300g 5통'
 '미소반 깔끔향긋 방풍나물절임 300g 5통' '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 2통'
 '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 3통' '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 5통'
 '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 5통' '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 5통'
 '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 5통' '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 2통'
 '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 2통' '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 3통'
 '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 2통' '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 3통'
 '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 5통' '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 3통'
 '미소반 환상궁합 고추냉이잎 절임 300g 입맛 돋우는 돌산갓 절임'
 '미소반 환상궁합 고추냉이잎 절임 300g 입맛 돋우는 돌산갓 절임'
 '미소반 환상궁합 고추냉이잎 절임 300g 입맛 돋우는 돌산갓 절임'
 '미소반 환상궁합 고추냉이잎 절임 300g 입맛 돋우는 돌산갓 절임'
 '미소반 환상궁합 고추냉이잎 절임 300g 입맛 돋우는 돌산갓

## 5-2. Sequential filter(img->text)

In [21]:
prod_filter = filter_handler.get_filter('sequential')
sim_titles = prod_filter.do(query_title, filter_type='img')

print('제목 : {0}'.format(query_title[0]))
print('-'*50)
print('유사 상품 제목')
print(sim_titles.flatten())

제목 : 미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 300g
--------------------------------------------------
유사 상품 제목
['미소반 환상궁합 고추냉이잎 절임 300g 입맛 돋우는 돌산갓 절임'
 '미소반 환상궁합 고추냉이잎 절임 300g 입맛 돋우는 돌산갓 절임'
 '미소반 환상궁합 고추냉이잎 절임 300g 입맛 돋우는 돌산갓 절임'
 '미소반 환상궁합 고추냉이잎 절임 300g 입맛 돋우는 돌산갓 절임'
 '미소반 환상궁합 고추냉이잎 절임 300g 입맛 돋우는 돌산갓 절임 300g'
 '[원더랜드] 미소반 환상궁합 고추냉이잎 절임 300g 입맛 돋우는 돌산갓 절임 300g'
 '미소반 환상궁합 고추냉이잎 절임 300g 입맛 돋우는 돌산갓 절임 300g'
 '미소반 환상궁합 고추냉이잎 절임 300g 입맛 돋우는 돌산갓 절임 300g'
 '고추냉이 냉이잎 냉이 미소반 환상궁합 고추냉이잎 절임 300g 입맛 돋우는 돌산갓'
 '미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 3'
 '미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 3'
 '미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 3'
 '미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 300g'
 '[원더랜드] 미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 300g'
 '미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 300g'
 '미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 300g'
 '상하식품 300g 환상궁합 300g 장아찌 절임 돋우는 국산 미소'
 '국산 절임 신선식품 방풍나물절임 돋우는 미소반 상하식품 300g '
 '절임 방풍나물절임 미소반 상하식품 300g 신선식품 반찬 장아찌 국산 돌산갓 깔끔향긋 돋우는'
 '국산 장아찌 상하식품 방풍나물절임 반찬 300g 절임 미소반 돌산갓 돋우는 깔끔향긋 신선식품'
 '환상궁합 국산 

## 5-3. Sequential filter(text->img)

In [22]:
prod_filter = filter_handler.get_filter('sequential')
sim_titles = prod_filter.do(query_title, filter_type='title')

print('제목 : {0}'.format(query_title[0]))
print('-'*50)
print('유사 상품 제목')
print(sim_titles.flatten())

제목 : 미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 300g
--------------------------------------------------
유사 상품 제목
['미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 2통' '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 3통'
 '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 5통' '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 5통'
 '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 5통' '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 5통'
 '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 2통' '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 2통'
 '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 3통' '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 2통'
 '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 3통' '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 5통'
 '미소반 톡소는 맛이 입맛 돋우는 돌산갓 절임 300g 3통' '미소반 환상궁합 고추냉이잎 절임 300g 입맛 돋우는 돌산갓 절임'
 '미소반 환상궁합 고추냉이잎 절임 300g 입맛 돋우는 돌산갓 절임'
 '미소반 환상궁합 고추냉이잎 절임 300g 입맛 돋우는 돌산갓 절임'
 '미소반 환상궁합 고추냉이잎 절임 300g 입맛 돋우는 돌산갓 절임'
 '미소반 환상궁합 고추냉이잎 절임 300g 입맛 돋우는 돌산갓 절임 300g'
 '미소반 환상궁합 고추냉이잎 절임 300g 입맛 돋우는 돌산갓 절임 300g'
 '미소반 환상궁합 고추냉이잎 절임 300g 입맛 돋우는 돌산갓 절임 300g'
 '미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 3'
 '미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 3'
 '미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 3'
 '미소반 입맛 돋우는 돌산갓 절임

## 5-4. Concat filter

In [23]:
prod_filter = filter_handler.get_filter('concat')
sim_titles = prod_filter.do(query_title)

print('제목 : {0}'.format(query_title[0]))
print('-'*50)
print('유사 상품 제목')
print(sim_titles.flatten())

제목 : 미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 300g
--------------------------------------------------
유사 상품 제목
['미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 3'
 '미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 3'
 '미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 3'
 '미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 300g'
 '[원더랜드] 미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 300g'
 '미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 300g'
 '미소반 입맛 돋우는 돌산갓 절임 300g 깔끔향긋 방풍나물절임 300g'
 '국산 절임 신선식품 방풍나물절임 돋우는 미소반 상하식품 300g '
 '절임 방풍나물절임 미소반 상하식품 300g 신선식품 반찬 장아찌 국산 돌산갓 깔끔향긋 돋우는'
 '국산 장아찌 상하식품 방풍나물절임 반찬 300g 절임 미소반 돌산갓 돋우는 깔끔향긋 신선식품'
 '국산 돌산갓 미소반 깔끔향긋 입맛 상하식품 돋우는 300g 방풍나']
